In [2]:
using MUST
using PyPlot
using PyCall

MUST.@import_dispatch "../../../dispatch2"
MUST.@import_dispatch "../../../dispatch2" EOS

PyObject <module 'EOS' from '/cobra/ptmp/peitner/model_grid/dispatch2/utilities/python/dispatch/EOS/__init__.py'>

In [3]:
sqEos = MUST.@squaregasPythonEOS 
leEos = MUST.@legacyPythonEOS 

MUST.PythonEOS(PyObject <EOS.stagger.stagger object at 0x2ab4c7f5ce20>, [:d, :ee], Bool[1, 0])

In [4]:
units = MUST.StaggerCGS()

MUST.AtmosUnits("cgs", 1.0e8, 1.0e-7, 100.0, 1.0e6, 1.0e17, 100000.0, 0.09999999999999999, 1.0e12, 100000.0, 0.1, 1.0e-8, 1.3, 1120.9982432795857, 1.380658e-16, 1.6726219e-24, 6.65e-24)

In [5]:
@show sqEos.eos.scale0.name sqEos.eos.scale0.min sqEos.eos.scale0.max
@show sqEos.eos.scale1.name exp(sqEos.eos.scale1.min) exp(sqEos.eos.scale1.max);

sqEos.eos.scale0.name = "ln(rho)"
sqEos.eos.scale0.min = -30.50869679124781
sqEos.eos.scale0.max = -5.147015827384369
sqEos.eos.scale1.name = "ln(ee)"
exp(sqEos.eos.scale1.min) = 1.9964450599999976e12
exp(sqEos.eos.scale1.max) = 5.797331800000005e13


In [6]:
sqEos.eos.variables

Dict{Any, Any} with 4 entries:
  0 => "P"
  2 => "Ne"
  3 => "rkap"
  1 => "T"

In [7]:
leEos.eos.variables

Dict{Any, Any} with 11 entries:
  5  => "T"
  8  => "S2"
  1  => "K1"
  0  => "P"
  6  => ""
  9  => "S3"
  3  => "K3"
  7  => "S1"
  4  => "K4"
  2  => "K2"
  10 => "S4"

In [8]:
e  = MUST.bisect(sqEos, ee=[exp(sqEos.eos.scale1.min), exp(sqEos.eos.scale1.max)], d=3.2599e-7, T=8580.2)
t  = MUST.lookup(sqEos, "T", 3.2599e-7, e)
p  = MUST.lookup(sqEos, "P", 3.2599e-7, e)
k  = MUST.lookup(sqEos, "rkap", 3.2599e-7, e)

@show e t p k;

e = 4.319760345445752e12
t = 8580.199999999812
p = 180844.34429145555
k = 10.246975093703007


In [9]:
e  = MUST.bisect(leEos, ee=[5,30], d=3.2599e-7 ./units.d, T=8580.2) 
t  = MUST.lookup(leEos, "T", 3.2599e-7./units.d, e)
p  = MUST.lookup(leEos, "P", 3.2599e-7./units.d, e) 
k  = MUST.lookup(leEos, "K1", 3.2599e-7./units.d, e) 

@show e t p k;

e = 5.761841710646727
t = 8580.199999999953
p = 1.8076766498782928
k = 486.19325453306675


In [10]:
units.rk

1.0e-8

In [11]:
units.l

1.0e8

In [12]:
@pyimport numpy

In [13]:
path = MUST.@in_dispatch "input_data/square_gas/eostable.dat"
data = numpy.memmap(path,mode="r",order="F",shape=(1024,500,4),dtype="<f4");

In [14]:
#using Mmap

In [15]:
#m = Mmap.mmap(path, Array{Float32,3}, (500,1024,4));

In [16]:
"""
Square Gas

d  4.0023014E-02 
ee   3.757261     
opacity  6.2012821E-03 
src  1.2659934E-26 
ne 1.1200634E-13


d  4.0126657E-03 
ee   3.757261     
opacity  1.8582983E-04 
src  5.9403183E-27 
ne 1.5834541E-14
"""

"Square Gas\n\nd  4.0023014E-02 \nee   3.757261     \nopacity  6.2012821E-03 \nsrc  1.2659934E-26 \nne 1.1200634E-13\n\n\nd  4.0126657E-03 \nee   3.757261     \nopacity  1.8582983E-04 \nsrc  5.9403183E-27 \nne 1.5834541E-14\n"

In [17]:
"""
Stagger

d  4.0023014E-02 
ee   5.200000     
opacity  6.9835219E-03 
src  4.5125615E-02 
ne 1.1510103E+11

d  4.0126657E-03 
ee   5.200000     
opacity  1.5603274E-04 
src  4.4788420E-02 
ne 1.5962885E+10
"""   

"Stagger\n\nd  4.0023014E-02 \nee   5.200000     \nopacity  6.9835219E-03 \nsrc  4.5125615E-02 \nne 1.1510103E+11\n\nd  4.0126657E-03 \nee   5.200000     \nopacity  1.5603274E-04 \nsrc  4.4788420E-02 \nne 1.5962885E+10\n"

In [18]:
MUST.lookup(sqEos, "T", 4.0023014E-02 *units.d, 3.757261 *units.ee)

4058.522259742569

In [19]:
MUST.lookup(leEos, "T", 4.0023e-2, 5.2)

4063.0608350731313

In [20]:
d_cgs = 4.0126657E-03 *units.d

4.0126656999999997e-10

In [21]:
src_cgs = 5.9403183E-27 * (units.e / units.t * units.l)

5.9403183e-16

In [22]:
4.4788420E-02 * (units.e / units.t * units.l)

4.478842e9

In [23]:
rk_cgs = 1.8582983E-04 * units.rk

1.8582983e-12

In [26]:
4.4788420E-02/4.0126657E-03 * (units.e / units.t * units.l)

1.116176211738745e12

In [24]:
"""
Square Gas
d  2.2079017E-04 
ee   3.757261     
opacity  4.6575392E-06  2.5362371E-05 2.3297449E-05  2.9776373E-04 
src  6.2964849E-17  6.6287261E-19  2.5442887E-19 5.7439292E-20
"""

"Square Gas\nd  2.2079017E-04 \nee   3.757261     \nopacity  4.6575392E-06  2.5362371E-05 2.3297449E-05  2.9776373E-04 \nsrc  6.2964849E-17  6.6287261E-19  2.5442887E-19 5.7439292E-20\n"

In [27]:
9.6016154E-32   * (units.e / units.t * units.l)

9.6016154e-21

In [28]:
units.e

100000.0

In [ ]:
sc. e  9.999999999999999E+028 
sc. t   100.000000000000      
sc. l 100000000.000000      
sc. kr  0.100000000000000  

In [31]:
units.m * units.l^2/units.t^2 #/ (units.l)^3

1.0e29

In [36]:
units.m

1.0e17